PyMuPDFテストコード

In [1]:
# import pymupdf
import fitz

# inputfile = "001安良川八軒屋地区堤防付属物設置工事.pdf"
inputfile = "①入札説明書_Ｒ６東京国道管内改築保全工事.pdf"

fin = open(inputfile, "rb")

#  fin = pymupdf.open(inputfile) # open a document
fin = fitz.open(inputfile) # open a document
out = open("test_output.txt", "wb") # create a text output
for page in fin: # iterate the document pages
    text = page.get_text().encode("utf-8")  # get plain text (is in UTF-8)
    out.write(text) # write text of page
    out.write(bytes((12,))) # write page delimiter (form feed 0x0C)
out.close()





In [2]:
# ８９文字化け対応
# cp932, euc_jp, shift_jis
# pymupdf モジュールの読み込み
import fitz


# inputfile = "001安良川八軒屋地区堤防付属物設置工事.pdf"
inputfile = "001安良川八軒屋地区堤防付属物設置工事-1.pdf"

fin = open(inputfile, "rb")

fin = fitz.open(inputfile) # open a document
out = open("test_output2.txt", "wb") # create a text output
for page in fin: # iterate the document pages
    text = page.get_text().encode("utf-8") # get plain text (is in utf-8)
    out.write(text) # write text of page
    out.write(bytes((12,))) # write page delimiter (form feed 0x0C)
out.close()





FileNotFoundError: [Errno 2] No such file or directory: '001安良川八軒屋地区堤防付属物設置工事-1.pdf'

# 【ここから各地整毎での文字列抽出のサブルーチン】
- 優先　（表外or文書内）８５、８７、８８、９０、８１＜開発建設部毎＞
- 要検討　（表内）８２、８３、８４、８６、８９

### ８５中部地整：テキスト文字列からの抽出「より同種性の高い工事」の内容抽出
- 抽出元：入札説明書（表外の表記より）

In [2]:
# 85中部地整：入札説明書（PDF）から「より同種性が高い」工事要件を抽出するテストプログラム
import re

# inputfile = "7-5_令和２年度　天竜川水系小嵐第３砂防堰堤工事.txt"
inputfile = "【公告・入札説明書】令和５年度　高山国道管内トンネル照明設備工事.txt"
# inputfile = "【公告・入札説明書】令和５年度　高山国道区画線維持工事.txt" #　！事務所名抽出出来ず！
# inputfile = "【入札公告・入札説明書】令和5年度 設楽ダム国道257号田口地区道路建設工事.txt"
# inputfile = "【公告・入札説明書】令和５年度　４１号高山国府トンネル換気設備整備工事.txt"
# inputfile = "静岡河川01_setsumeisyo.txt"
# inputfile = "説明書　令和５年度　設楽ダム本体建設第１期工事.txt"
# inputfile = "説明書　令和５年度 一般県道松原芋島線川島大橋橋脚工事.txt"
# inputfile = "説明書　令和５年度　１号清水立体尾羽第１高架橋鋼上部工事　他.txt"

with open(inputfile, "rt", encoding="utf-8") as f:  # 改行コード等除去後のテキストファイルを開く
  text = f.read()

print(inputfile)
print(len(text))
print("総合評価に" in text)

# 入札公告が先頭部部分にある場合、入札説明書のページ以降を抽出する
if (("入札公告（建設工事）" in text) and text.find("入札公告（建設工事）") <20 ):
  t = text.partition("④  開札日時")  # 入札公告の最終ページの記載文をタプルとして抽出
  text = t[-1]                       # 以降の文字列を抽出
  t = text.partition("入札説明書")   # 入札説明書の１ページ目の記載文
  text = t[-1]                       # 以降の文字列を抽出（入札説明書の部分（行頭欠け））

print(len(text))

# 整備局等名・事務所名の抽出
# namu_bu 地方整備局等
# name_of 事務所名（本館工事の場合は「本官」）

if ("中部地方整備局" in text):
  name_bu ="中部地方整備局"

print(name_bu)

# print(("支出負担行為担当官 中部地方整備局長" in text))  # 本官と分任官工事とのルーチン分け
if ("支出負担行為担当官 中部地方整備局長" in text):  # 本官と分任官とのルーチン分け
  name_of = "本官"
else:  # 事務所名抽出：２つめの分任支出負担行為担当官以下の　所長までの部分でスライス
  t0 = text.partition("分任支出負担行為担当官")  # 入札説明書の１つめの同一文字列の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("分任支出負担行為担当官") # 入札説明書の２つめの同一文字列の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("中部地方整備局")        # ”中部地方整備局”の前半をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("所長")                  # "所長"の文字列の前をタプルとして抽出
  text0 = t0[0]
  name_of = text0.strip() + "所"              # 冒頭空白文字を削除して、抽出した文字列に"所"を追加

print(name_of)

# 工事名の抽出
# name_co 工事名
# (1) 工事名 , (2) 工事場所 で文字列を抽出スライス

t0 = text.partition("工事名")  # "工事名"の文字列の後をタプルとして抽出
text0 = t0[-1]
t0 = text0.partition("(2)")    # 次行にある(2)の文字列の前をタプルとして抽出
text0 = t0[0]
t0 = text0.partition("（電子入札対象案件）")  # "（電子入札対象案件）"文字列の前をタプルとして抽出
text0 = t0[0]
name_co =text0.strip()         # 前後の空白文字を削除

print(name_co)
print("総合評価に関する事項" in text)

# 同種工事の内容　入札説明書　４）競争参加資格の項目より抽出　【企業】同種工事、【技術者】同種工事
# doushu_co 企業での同種工事の内容
# doushu_en 技術者での同種工事の内容
# 本官工事での場合で【企業】【技術者】の文字列がない場合は「特殊工事のため要確認」を入力
# 要検討：正規表現での抽出

if name_of != "本官": # 本官以外の場合
  t0 = text.partition("元請けとして、以下に示す同種工事")  #"元請けとして、以下に示す同種工事"の後の文字列をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("【企業】") # "【企業】"の後の文字列をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("同種工事：") # "同種工事："の後の文字列をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("【技術者】") # "【技術者】"の前の文字列をタプルとして抽出
  text0 = t0[0]
  doushu_co = text0.strip() # 前後の空白文字を削除
  print("「同種工事（企業）」", doushu_co)
  text0 = t0[-1]  # "技術者"の後の文字列を入力
  t0 = text0.partition("同種工事：")  # "同種工事："の後の文字列をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("(5)") # (5)の前の文字列をタプルとして抽出
  text0 = t0[0]
  doushu_en = text0
  print("「同種工事（技術者）」", doushu_en)

elif (("【企業】" in text) and ("【技術者】") in text): # 抽出する文字列２つがある場合のサブルーチン
  t0 = text.partition("発注者から企業に対して通知された評定点が")    # 文字列の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("【企業】") # "【企業】"の後の文字列をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("同種工事：") # "同種工事："の後の文字列をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("【技術者】") # "【技術者】"の前の文字列をタプルとして抽出
  text0 = t0[0]
  doushu_co = text0.strip() # 前後の空白文字を削除
  print("「同種工事（企業）」", doushu_co)
  text0 = t0[-1]  # "技術者"の後の文字列を入力
  t0 = text0.partition("同種工事：")  # "同種工事："の後の文字列をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("(5)") # (5)の前の文字列をタプルとして抽出
  text0 = t0[0]
  t0 = text0.partition("(6)") # (6)の前の文字列をタプルとして抽出
  text0 = t0[0]
  doushu_en = text0.strip()
  print("「同種工事（技術者）」", doushu_en)

else:  # 企業・技術者の表記がない場合のサブルーチン
  t0 = text.partition("発注者から企業に対して通知された評定点が")    # 文字列の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("同種工事：") # "同種工事："の後の文字列をタプルとして抽出
  text0 = t0[-1]
  print(text0)
  t0 = text0.partition("(6)") # (6)の前の文字列をタプルとして抽出
  text0 = t0[0]
  doushu_co = text0.strip() # 前後の空白文字を削除
  print("「同種工事（企業）」", doushu_co)
  doushu_en = "特殊工事のため要確認"
  print("「同種工事（技術者）」", doushu_en)


# 「より同種性が高い」工事の要件抽出
# douahu01 str
# (3) 入札の評価に関する基準及び得点配分
# 工事の総合評価に関する加算点の表外にある記載「※１：同種工事の内・・・・を「より同種性が高い」の部分文字列を抽出

if (("※１：" in text) and ("を「より同種性が高い」と評価" in text)):  # 記載があるかの判定
  t1 = text.partition("※１：")   # "※１："の文字列の後
  text1 = t1[-1]
#  print(t1[-1])
  t1 = text1.partition("同種工事のうち、")
  text1 =t1[-1]
#  print(t2[-1])
  t1 = text1.partition("を「より同種性が高い」と評価")
  text1 =t1[0]
else:
  text1 = "該当なし"

doushu01 = text1
print("「より同種性の高い」", doushu01)


【公告・入札説明書】令和５年度　高山国道管内トンネル照明設備工事.txt
95082
True
86731
中部地方整備局
高山国道事務所
令和５年度高山国道管内トンネル照明設備工事
False
「同種工事（企業）」 道路照明（トンネル照明を含む）を設置（新設、増設、改設又は更新）した工事の施工実績
「同種工事（技術者）」 道路照明（トンネル照明を含む）を設置（新設、増設、改設又は更新）した工事の施工実績 
「より同種性の高い」 交通規制（通行止めを除く）を伴う、トンネル照明を設置（新設、増設、改設又は更新）した工事の施工実績（企業の能力のみ）


### ８６近畿地整：テキスト文字列からの抽出「同種性」
- 入札説明書_別紙-5　<.xls or .pdf> besshi-5.pdf、入札説明書<.doc or .pdf>
- 別紙-5のファイル名で集約する必要あり

### ８７中国地整：テキスト文字列からの抽出「同種性」
- 抽出元：入札説明書（表外の表記より）
- 河川・道路でテンプレが若干違う可能性有り

In [ ]:
# 87中国地整：入札説明書（PDF）から「より同種性が高い」工事要件を抽出するテストプログラム
import re


# inputfile = "230619-松江-01説明書.txt"
# inputfile = "【訂正230531】230526-鳥取-03説明書.txt"
# inputfile = "230613-広島国道ｰ02入札説明書.txt"
# inputfile = "230512-本官-01入札説明書.txt"
inputfile = "230607-太田川-入札説明書.txt"

with open(inputfile, "rt", encoding="utf-8") as f:  # 改行コード等除去後のテキストファイルを開く
  text = f.read()

print(inputfile)
print(len(text))
print("総合評価に関する事項" in text)

# 整備局等名・事務所名の抽出
# namu_bu 地方整備局等
# name_of 事務所名（本官工事の場合は「本官」）

if ("中国地方整備局" in text):
  name_bu ="中国地方整備局"

print(name_bu)

print(("支出負担行為担当官" in text) and ("中国地方整備局長" in text))  # 本官と分任官工事とのルーチン分け
if ("支出負担行為担当官中国地方整備局長" in text):  # 本官と分任官とのルーチン分け
  name_of = "本官"
else:  # 事務所名抽出：１つめの分任支出負担行為担当官以下の　所長までの部分でスライス
  t0 = text.partition("分任支出負担行為担当官")  # 入札説明書の１つめの同一文字列の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("中国地方整備局")        # ”中国地方整備局”の前半をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("所長")                  # "所長"の文字列の前をタプルとして抽出
  text0 = t0[0]
  name_of = text0.strip() + "所"              # 冒頭空白文字を削除して、抽出した文字列に"所"を追加

print(name_of)

# 工事名の抽出
# name_co  工事名 
# (1) 工事名 , (2) 工事場所 で文字列を抽出スライス

t0 = text.partition("工事名")  # "工事名"の文字列の後をタプルとして抽出
text0 = t0[-1]
t0 = text0.partition("(2)")    # 次行にある(2)の文字列の前をタプルとして抽出
text0 = t0[0]
t0 = text0.partition("（電子入札対象案件）")  # "（電子入札対象案件）"文字列の前をタプルとして抽出
text0 = t0[0]
name_co =text0.strip()         # 前後の空白文字を削除

print(name_co)

#　同種工事の要件文字列の抽出（企業） doushu_co
if (("同種工事とは、" in text) and ("なお、同種工事の施工実績は、" in text)):  # 記載があるかの判定
  t0 = text.partition("同種工事とは、")   # "同種工事とは、"の後の文字列をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("なお、同種工事の施工実績は、") # 文字列の前をタプルとして抽出
  text0 =t0[0]
  doushu_co = text0.strip() # 前後の空白文字を削除

#　同種工事の要件文字列の抽出（技術者） doushu_en
if ("現場での作業に配置する技術者は、" in text):  # 記載があるかの判定
  t0 = text.partition("現場での作業に配置する技術者は、")  # "現場での作業に配置する技術者は、"の文字列の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("同種工事とは、") # 文字列の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("ただし、") # 文字列の前をタプルとして抽出
  text0 = t0[0]
  doushu_en = text0.strip() # 前後の空白文字を削除 
elif ("基準を満たす主任技術者又は監理技術者を当該工事に" in text):  # 記載があるかの判定
  t0 = text.partition("基準を満たす主任技術者又は監理技術者を当該工事に")  # 文字列の後をタプルとして抽出
  text0 = t0[-1]
  if (("に掲げる同種工事の経験を有する者であること。" in text0) or 
      ("の同種工事の施工実績を有していること。" in text0)): # 企業部分と同種工事の技術者実績の記載があるかの判定
    doushu_en = doushu_co
  
print("「同種工事（企業）」", doushu_co)
print("「同種工事（技術者）」", doushu_en)


# より同種性の高い／同種性の高い工事の内容の抽出
# 5. 技術的能力の審査及び総合評価に関する事項 において、
# 下記の表記が箇条書き表記で見られる場合は、リストに降順に入力
# doushusei_co[0] 企業での「より高い同種性が認められる工事」の内容
# doushusei_co[1] 企業での「高い同種性が認められる工事」の内容
# doushusei_co[2] 企業での「同種性が認められる工事」の内容
# doushusei_en[0] 技術者での「より高い同種性が認められる工事」の内容
# doushusei_en[1] 技術者での「高い同種性が認められる工事」の内容
# doushusei_en[2] 技術者での「同種性が認められる工事」の内容
# score_co 企業での同種工事の施工実績での配点点数 (str)  　# 要検討：数値化してリスト化
# score_en 技術者での同種工事の施工実績での配点点数 (str)　# 要検討：数値化してリスト化
# 上記表記が見られず、加算点を与える文章表現（例）3.0点／0.0点））に付随した表記の場合は
# その文字列部分を doususei_co[0] doushusei_en[0]に入力する
# 要検討：正規表現での抽出


doushusei_co = list()
doushusei_en = list()

if name_of != "本官": # 本官以外の場合
  if (("より高い同種性が認められる工事：" in text) or ("高い同種性が認められる工事：" in text)):
    # 上記文字列で箇条書き記載がある場合の処理ルーチン 
    t0 = text.partition("企業の能力等（加算点）で提出した企業において") # 後の文字列をタプルとして抽出
    text0 = t0[-1]
    t0 = text0.partition("企業の能力等（加算点）")  #"企業の能力等（加算点）"の後の文字列をタプルとして抽出
    text0 = t0[-1]
    t1 = text0.partition("で評価し、それぞれ") # 後の文字列をタプルとして抽出
    text1 = t1[-1]
    t1 = text1.partition("の加算点を") # 前の文字列をタプルとして抽出
    text1 = t1[0]
    score_co = text1.strip() # 前後の空白文字を削除
    
    if ("より高い同種性が認められる工事：" in text0):
      t0 = text0.partition("より高い同種性が認められる工事：")  # "より高い同種性が認められる工事："の後の文字列をタプルとして抽出
      text0 = t0[-1]
      t1 = text0.partition("より高い同種性が認められる工事の実績")  # 前の文字列をタプルとして抽出
      text1 = t1[0]
      doushusei_co.insert(0, text1)
    doushusei_co.insert(1, "")
    if ("同種性が認められる工事：" in text0):
      t0 = text0.partition("同種性が認められる工事：")  # "同種性が認められる工事："の後の文字列をタプルとして抽出
      text0 = t0[-1]
      t1 = text0.partition("共同企業体の構成員")  # 前の文字列をタプルとして抽出
      text1 = t1[0]
      doushusei_co.insert(2, text1)
    
    t0 = text0.partition("技術者の能力等（加算点）")  #"技術者の能力等（加算点）"の後の文字列をタプルとして抽出
    text0 = t0[-1]
    t1 = text0.partition("で評価し、それぞれ") # 後の文字列をタプルとして抽出
    text1 = t1[-1]
    t1 = text1.partition("の加算点を") # 前の文字列をタプルとして抽出
    text1 = t1[0]
    score_en = text1.strip() # 前後の空白文字を削除

    if ("より高い同種性が認められる工事：" in text0):
      t0 = text0.partition("より高い同種性が認められる工事：")  # "より高い同種性が認められる工事："の後の文字列をタプルとして抽出
      text0 = t0[-1]
      t1 = text0.partition("より高い同種性が認められる工事の実績")  # 前の文字列をタプルとして抽出
      text1 = t1[0]
      doushusei_en.insert(0, text1)
    if ("高い同種性が認められる工事：" in text0):
      t0 = text0.partition("同種性が認められる工事：")  # "同種性が認められる工事："の後の文字列をタプルとして抽出
      text0 = t0[-1]
      t1 = text0.partition("高い同種性が認められる工事の実績")  # 前の文字列をタプルとして抽出
      text1 = t1[0]
      doushusei_en.insert(1, text1)
    if ("同種性が認められる工事：" in text0):
      t0 = text0.partition("同種性が認められる工事：")  # "同種性が認められる工事："の後の文字列をタプルとして抽出
      text0 = t0[-1]
      t1 = text0.partition("共同企業体の構成員")  # 前の文字列をタプルとして抽出
      text1 = t1[0]
      doushusei_en.insert(2, text1)

  else: # 配点表記とともに文章内に内容記載がある場合の処理ルーチン
    t0 = text.partition("企業の能力等（")    # "企業の能力等（"文字列の後をタプルとして抽出
    text0 = t0[-1]
    t1 = text0.partition("同種工事の実績において、") # "同種工事の実績において、"の後の文字列をタプルとして抽出
    text1 = t1[-1]
    t1 = text1.partition("で評価し、") # "で評価し、"の前の文字列をタプルとして抽出
    text1 = t1[0]
    doushusei_co.insert(0,text1)
    t0 = text0.partition("、それぞれ")  # "、それぞれ"の後の文字列をタプルとして抽出
    text1 = t0[-1]
    t1 = text1.partition("の加算点を与える。")  # "の加算点を与える。"の前の文字列をタプルとして抽出
    text1 = t1[0]
    score_co = text1

    t0 = text0.partition("技術者の能力等（") # "配置予定技術者の能力等（"の後の文字列をタプルとして抽出
    text0 = t0[-1]
    if ("補佐の場合において、" in text0):
      t1 = text0.partition("補佐の場合において、")  # "補佐の場合において、"の後の文字列をタプルとして抽出
      text1 = t1[-1]
    if("補助者の場合において、" in text0):
      t1 = text0.partition("補助者の場合において、") # "補助者の場合において、"の後の文字列をタプルとして抽出
      text1 = t1[-1]
    t1 = text1.partition("で評価し、") # "で評価し、"の前の文字列をタプルとして抽出
    text1 = t1[0]
    doushusei_en.insert(0,text1)
    t1 = text0.partition("で評価し、") # "で評価し、"の後の文字列をタプルとして抽出
    text1 = t1[-1]
    t1 = text1.partition("の加算点を与える。") # 前の文字列をタプルとして抽出
    text1 = t1[0]
    text1 = text1.replace("それぞれ","")  # "それぞれが"が抽出文字列に入っている場合の削除処理"
    score_en = text1

else:  # 本官工事のサブルーチン
  # 企業の能力等（加算点）において記載のある本官工事を探す必要あり！！
  t0 = text.partition("技術者の能力等（")    # 文字列の後をタプルとして抽出
  text0 = t0[-1]
  if ("補佐の場合において、" in text0):
    t1 = text0.partition("補佐の場合において、")  # "補佐の場合において、"の後の文字列をタプルとして抽出
  if("補助者の場合において、" in text0):
    t1 = text0.partition("補助者の場合において、") # "補助者の場合において、"の後の文字列をタプルとして抽出
  if("当技術者の場合において、" in text0):
    t1 = text0.partition("担当技術者の場合において、") # "担当技術"の後の文字列をタプルとして抽出
  text1 = t1[-1]
  t1 = text1.partition("で評価し、") # "で評価し、"の前の文字列をタプルとして抽出
  text1 = t1[0]
  doushusei_en.insert(0,text1)
  t1 = text0.partition("で評価し、") # "で評価し、"の後の文字列をタプルとして抽出
  text1 = t1[-1]
  t1 = text1.partition("の加算点を与える。") # 前の文字列をタプルとして抽出
  text1 = t1[0]
  text1 = text1.replace("それぞれ","")  # "それぞれが"が抽出文字列に入っている場合の削除処理"
  score_en = text1


print("「より高い同種性／高い同種性／同種性（企業）list」", doushusei_co)
print("「配点（企業）」", score_co)
print("「より高い同種性／高い同種性／同種性（技術者）list」", doushusei_en)
print("「配点（技術者）」", score_co)


### ８８四国地整：テキスト文字列からの抽出「同種性」
- 抽出元：入札説明書（表外の表記より）
- 同種性での点数配分は表内表記のため、別途抽出ルーチンを作る必要あり

In [ ]:
# 88四国地整：入札説明書（PDF）から「より同種性が高い」工事要件を抽出するテストプログラム
import re


inputfile = "02 入札説明書【令和5年度　山鳥坂ダム月野尾工事用道路工事】.txt"
# inputfile = "02-1 入札説明書【R5 早明浦ダム周辺環境整備工事】.txt"
# inputfile = "02-1入札説明書【R5 香川管内安全施設工事】.txt"
# inputfile = "1101_02 入札説明書【令和5年度　愛南地区舗装修繕工事】.txt"
# inputfile = "0601_01-1 入札説明書.txt"
# inputfile = "0711_02 入札説明書本文.txt"
# inputfile = "0502_02-1 入札説明書.txt"


with open(inputfile, "rt", encoding="utf-8") as f:  # 改行コード等除去後のテキストファイルを開く
  text = f.read()

print(inputfile)
print(len(text))
print("総合評価に関する評価項目" in text)

# 整備局等名・事務所名の抽出
# namu_bu 地方整備局等
# name_of 事務所名（本官工事の場合は「本官」）

if ("四国地方整備局" in text):
  name_bu ="四国地方整備局"

print(name_bu)

print(("支出負担行為担当官" in text) and ("四国地方整備局長" in text))  # 本官と分任官工事とのルーチン分け
if ("支出負担行為担当官四国地方整備局長" in text):  # 本官と分任官とのルーチン分け
  name_of = "本官"
else:  # 事務所名抽出：１つめの分任支出負担行為担当官以下の　所長までの部分でスライス
  t0 = text.partition("分任支出負担行為担当官")  # 入札説明書の１つめの同一文字列の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("四国地方整備局")        # ”四国地方整備局”の前半をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("所長")                  # "所長"の文字列の前をタプルとして抽出
  text0 = t0[0]
  name_of = text0.strip() + "所"              # 冒頭空白文字を削除して、抽出した文字列に"所"を追加

print(name_of)

# 工事名の抽出
# name_co  工事名 
# (1) 工事名 , (2) 工事場所 で文字列を抽出スライス

t0 = text.partition("工事名")  # "工事名"の文字列の後をタプルとして抽出
text0 = t0[-1]
t0 = text0.partition("(2)")    # 次行にある(2)の文字列の前をタプルとして抽出
text0 = t0[0]
t0 = text0.partition("（電子入札及び電子契約対象案件）")  # "（電子入札対象案件）"文字列の前をタプルとして抽出
text0 = t0[0]
name_co =text0.strip()         # 前後の空白文字を削除

print(name_co)

#　同種工事の要件文字列の抽出（企業） doushu_co

if ("下記の条件を満足する同種工事を施工した実績を有すること" in text) :  # 記載があるかの判定
  t0 = text.partition("下記の条件を満足する同種工事を施工した実績を有すること")   # 後の文字列をタプルとして抽出
  text0 = t0[-1]
  if ("実績に限る。・同種工事として、" in text0):  # "・"以降の表記ゆれ対応
    t1 = text0.partition("実績に限る。・同種工事として、")   # 後の文字列をタプルとして抽出
  elif ("実績に限る。同種工事とは、" in text0): # "・"以降の表記ゆれ対応
    t1 = text0.partition("実績に限る。同種工事とは、")   # 後の文字列をタプルとして抽出
  else:
    t1 = text0.partition("実績に限る。・") # 後の文字列をタプルとして抽出
  text1 = t1[-1]
  t1 = text1.partition("なお、当該実績が") # 文字列の前をタプルとして抽出 
  text1 =t1[0]
  doushu_co = text1.replace("を有すること。","") # ”を有すること。”を空文字置換
elif ("下記の条件を満足する同種工事１を施工した実績を有すること" in text): 
  # 企業の実績を同種工事１としている記述への対応（本官工事）
  t0 = text.partition("下記の条件を満足する同種工事１を施工した実績を有すること")   # 後の文字列をタプルとして抽出
  text0 = t0[-1]
  t1 = text0.partition("同種工事１とは、") # 後の文字列をタプルとして抽出
  text1 = t1[-1]
  t1 = text1.partition("なお、当該実績は") # 文字列の前をタプルとして抽出
  text1 = t1[0]
  doushu_co = text1.replace("を有すること。","") # ”を有すること。”を空文字置換

#　同種工事の要件文字列の抽出（技術者） doushu_en

if name_of != "本官": # 分任官工事・本官工事とのサブルーチン分け
  if ("同種工事（上記（４）に掲げる工事）の経験を有する者であること" in text):  # 記載があるかの判定
    doushu_en = doushu_co 
  else:
    t1 = text0.partition("元請けの技術者として") # 後の文字列をタプルとして抽出
    text1 = t1[-1]
    t1 = text1.partition("（共同企業体の構成員としての経験は") # 文字列の前をタプルとして抽出
    text1 = t1[0]
    doushu_en = text1.replace("を有すること。", "")
elif ("同種工事２の経験を有する者" in text):
    # 技術者の実績を同種工事２としている記述への対応（本官工事）
    t1 = text0.partition("同種工事２とは") # 後の文字列をタプルとして抽出
    text1 = t1[-1]
    t1 = text1.partition("なお、当該経験は") # 文字列の前をタプルとして抽出
    text1 = t1[0]
    doushu_en = text1.replace("を有すること。", "")

print("「同種工事（企業）」", doushu_co)
print("「同種工事（技術者）」", doushu_en)


# より同種性の高い工事／同種性が認められる工事の内容の抽出
# （２）入札の評価に関する基準　の項目において
# 表外の記載からそれぞれの内容を抽出する
# doushusei_en[0] 技術者での「より高い同種性が認められる工事」の内容
# doushusei_en[1] 技術者での「同種性が認められる工事」の内容　≒同種工事
# doushusei_co[0] 企業での「より高い同種性の高い工事」の内容
# doushusei_co[1] 企業での「同種性が認められる工事」の内容　≒同種工事 
# score_en 技術者での同種工事の施工実績での配点点数 (str)　# 要検討：表内抽出・リスト化
# score_co 企業での同種工事の施工実績での配点点数 (str)  　# 要検討：表内抽出・リスト化

doushusei_co = list()
doushusei_en = list()

if name_of != "本官": # 本官以外の場合
  if (("「より同種性の高い工事」：" in text) or ("「同種性が認められる工事」" in text)):
    # 上記文字列で表外記載がある場合の処理ルーチン 
    t0 = text.partition("１）技術者評価") # "１）技術者評価"後の文字列をタプルとして抽出
    text0 = t0[-1]
    t1 = text0.partition("」における「より同種性の高い工事」とは、")  #"「より同種性の高い工事」とは、"の後の文字列をタプルとして抽出
    text1 = t1[-1]
    if ("を有することとする。" in text1):
      t1 =text1.partition("を有することとする。") # 表記揺れ対応
    else:
      t1 = text1.partition("とする。") # 前の文字列をタプルとして抽出
    text1 = t1[0]
    doushusei_en.insert(0, text1) 
    t1 = text0.partition("「同種性が認められる工事」とは、") # 後の文字列を抽出
    text1 = t1[-1]
    if ("記載している同種工事を示す。" in text1):
      doushusei_en.insert(1, doushu_en)
    # 同種工事の施工経験・表内の配点文字列の抽出 （要検討：表内抽出）
    # t0 = text0.partition("技より同種性の高い") # 後の文字列をタプルとして抽出
    # text0 = t0[-1]
    # t1 = text0.partition("術者等工事") # 前の文字列をタプルとして抽出
    # text1 = t1[0]
    # score_en = text1
    # t1 = text0.partition("同種性が認められ") # 後の文字列をタプルとして抽出
    # text1 = t1[-1]
    # t1 = text1.partition("る工事担当技術者") # 前の文字列をタプルとして抽出
    # text1 = t1[0]
    # score_en = score_en + " " + text1
    
    t0 = text0.partition("．企業の施工実績") # 後の文字列をタプルとして抽出
    text0 = t0[-1]
    t1 = text0.partition("示す。「より同種性の高い工事」とは、") # 後の文字列をタプルとして抽出
    text1 = t1[-1]
    if ("を有することとする。" in text1): # 表記ゆれ対応
      t1 = text1.partition("を有することとする。")
    else:  
      t1 = text1.partition("とする。") # 前の文字列をタプルとして抽出
    text1 = t1[0]
    doushusei_co.insert(0, text1)
    t1 = text0.partition("「同種性が認められる工事」とは、")
    text1 = t1[-1]
    if ("記載している同種工事を示す。" in text1):
      doushusei_co.insert(1, doushu_co)

    # 同種工事の施工実績・表内の配点文字列の抽出 （要検討：表内抽出・リスト化）
    # t1 = text0.partition("より同種性の高い工事の実績")
    # text1 = t1[-1]
    # t1 = text1.partition("／")
    # text1 = t1[0]
    # score_co = text1
    # t1 = text0.partition("同種性が認められる工事の実績")
    # text1 = t1[-1]
    # t1 = text1.partition("Ⅰ－ⅱ工事成績評定")
    # text1 = t1[0]
    # score_co = score_co + " " + text1
elif ("同種工事２の経験を有する者" in text):
  # 技術者の実績を同種工事２としている記述への対応（本官工事）
  t0 = text.partition("１）技術者評価")
  text0 = t0[-1]
  t1 = text0.partition("「より同種性の高い工事」とは、同種工事２のうち「")  # 後の文字列をタプルとして抽出
  text1 = t1[-1]
  t1 = text1.partition("」とする。") # 前の文字列をタプルとして抽出
  text1 = t1[0]
  doushusei_en.insert(0, text1) 
  t1 = text0.partition("「同種性が認められる工事」とは、") # 後の文字列を抽出
  text1 = t1[-1]
  if ("記載している同種工事２を示す。" in text1):
    doushusei_en.insert(1, doushu_en)
  # 企業の実績を同種工事１としている記述への対応（本官工事）
  t0 = text0.partition("．企業の施工実績") # 後の文字列を抽出
  text0 = t0[-1]
  t1 = text0.partition("「より同種性の高い工事」とは、同種工事１のうち「")
  text1 = t1[-1]
  t1 = text1.partition("」とする。")
  text1 = t1[0]
  doushusei_co.insert(0, text1)
  t1 = text0.partition("「同種性が認められる工事」とは、")
  text1 = t1[-1]
  if ("記載している同種工事１を示す。" in text1):
    doushusei_co.insert(1, doushu_co)



print("「より同種性の高い（企業）」", doushusei_co[0])
print("「同種性が認められる（企業）」", doushusei_co[1])
# print("「同種性配点（企業）」", score_co)    
print("「より同種性の高い（技術者）」", doushusei_en[0])
print("「同種性が認められる（技術者）」", doushusei_en[1])
# print("「同種性配点（技術者）」", score_en)


### ８９九州地整：テキスト文字列からの抽出「同種性」
- 抽出元：入札説明書（表外の表記より）
- PyMuPDFでUTF-8エンコード指定で文字化け　対応策検索中　→対応出来ないようなら、対象PDF類をコンバータで変換（txt or pdf）
- cp932, euc_jp, shift_jis

### ９０沖縄総合開発事務所：テキスト文字列からの抽出「同種性」
- 抽出元：入札説明書（表外の表記より）

In [ ]:
#90 沖縄総合事務局：入札説明書（PDF）から「より同種性が高い」工事要件を抽出するテストプログラム
import re


inputfile = "【入札説明書】令和5年祖宜名真情報板外設置工事.txt"
# inputfile = "【入札説明書】令和５年度　海洋博公園電気設備改修工事.txt"
# inputfile = "【入札説明書】令和5年度安波ダム系管理用制御処理設備設置工事.txt"
# inputfile = "【入札説明書】令和５年度羽地地区交通安全対策工事.txt"
# inputfile = "【入札説明書】令和5年度恩納BP４号橋下部工(下りA1・P1)工事.txt"
# inputfile = "【入札説明書】令和５年度恩納BP６号橋上部工（下りA1～P4）工事.txt"
# inputfile = "【入札説明書】令和5年度恩納南ＢＰ改良工事.txt"
# inputfile = "【入札説明書】令和５年度宜名真トンネルラジオ再放送設備設置工事.txt"
# inputfile = "【入札説明書】令和５年度国道３３２号植栽整備（その１）工事.txt"
# inputfile = "【入札説明書】令和５年度佐敷地区安全施設設置工事.txt"
# inputfile = "【入札説明書】令和５年度前兼久南地区電線共同溝設置工事.txt"
# inputfile = "【入札説明書】令和5年度北部国道管内交通安全対策工事.txt"
# inputfile = "【入札説明書】令和5年度北部国道管内保全(その１)工事.txt"


with open(inputfile, "rt", encoding="utf-8") as f:  # 改行コード等除去後のテキストファイルを開く
  text = f.read()

print(inputfile)
print(len(text))
print("総合評価に関する事項" in text)

# 整備局等名・事務所名の抽出
# namu_bu 地方整備局等
# name_of 事務所名（本官工事の場合は「本官」）

if ("沖縄総合事務局" in text):
  name_bu =""

print(name_bu)

print ("支出負担行為担当官沖縄総合事務局開発建設部長" in text)  # 本官と分任官とのルーチン分け
if ("支出負担行為担当官沖縄総合事務局開発建設部長" in text):  # 本官と分任官とのルーチン分け
  name_of = "本官"
else:  # 事務所名抽出：１つめの分任支出負担行為担当官以下の　所長までの部分でスライス
  t0 = text.partition("分任支出負担行為担当官")  # 入札説明書の１つめの同一文字列の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("沖縄総合事務局")        # ”四国地方整備局”の前半をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("所長")                  # "所長"の文字列の前をタプルとして抽出
  text0 = t0[0]
  name_of = text0.strip() + "所"              # 冒頭空白文字を削除して、抽出した文字列に"所"を追加

print(name_of)

# 工事名の抽出
# name_co  工事名 
# (1) 工事名 , (2) 工事場所 で文字列を抽出スライス

t0 = text.partition("工事名")  # "工事名"の文字列の後をタプルとして抽出
text0 = t0[-1]
t0 = text0.partition("(2)")    # 次行にある(2)の文字列の前をタプルとして抽出
text0 = t0[0]
t0 = text0.partition("（電子入札対象案件）")  # "（電子入札対象案件）"文字列の前をタプルとして抽出
text0 = t0[0]
name_co =text0.strip()         # 前後の空白文字を削除

print(name_co)

#　同種工事の要件文字列の抽出（企業） doushu_co

if (("「同種工事」は、次の要件を満たす" in text) and ("同種工事とは４．（４）に明示" in text)):
   # 「「同種工事」は、次の要件を満たす施工実績を有すること」記述への対応（公園事務所のＣＣＴＶ工事）
  t0 = text.partition("次に掲げる施工実績を有すること。")   # 後の文字列をタプルとして抽出
  text0 = t0[-1]
  if ("「同種工事」は" in text):
    t1 = text0.partition("「同種工事」は、次の要件を満たす施工実績を有すること。・")
  t1 = text1.partition("※施工実績が確認できる資料(コリンズ") # 文字列の前をタプルとして抽出 
  text1 =t1[0]

elif ("・同種工事" in text) and ("・より同種工事" in text):  # ・同種工事　・より同種工事と箇条書き記述への対応
  t0 = text.partition("次の要件を満たす施工実績を有すること。")
  text0 = t0[-1]
  t1 = text0.partition("・同種工事：") # 後の文字列をタプルとして抽出
  text1 = t1[-1]
  t1 = text1.partition("・より同種工事：") # 文字列の前をタプルとして抽出
  text1 = t1[0]

elif ("同種工事：" in text): # 同種工事：との記述への対応
  t0 = text.partition("次の要件を満たす施工実績を有すること。")
  text0 = t0[-1]
  t1 = text0.partition("同種工事：") # 後の文字列をタプルとして抽出
  text1 = t1[-1]
  t1 = text1.partition("※コリンズで") # 文字列の前をタプルとして抽出
  text1 = t1[0]

elif (("次の(ｱ)、(ｲ)の要件を満たす施工実績を有すること。" in text) and ("同種工事とは４．（４）に明示" in text)):
  # 同種工事として(ｱ)、(ｲ)との箇条書き記述への対応
  t0 = text.partition("次に掲げる施工実績を有すること。")
  text0 = t0[-1] # 後の文字列をタプルとして抽出
  t1 = text0.partition("(ｲ)の要件を満たす施工実績を有すること。")
  text1 = t1[-1] # 後の文字列をタプルとして抽出
  t1 = text1.partition("※コリンズで")  # 文字列の前をタプルとして抽出
  text1 = t1[0]

elif (("次の要件を満たす施工実績を有すること" in text) and ("同種工事とは４．（４）に明示" in text)) :  # 記載があるかの判定
  t0 = text.partition("次の要件を満たす施工実績を有すること")   # 後の文字列をタプルとして抽出
  text0 = t0[-1]
  if ("同種工事：" in text0):  # 同種工事と箇条書き記述への対応
    t1 = text0.partition("同種工事：" in text0) # 後の文字列をタプルとして抽出
    text1 = t1[-1]
    t1 = text1
  else:
    t1 = text0.partition("・") # 後の文字列をタプルとして抽出
    text1 = t1[-1]
  t1 = text1.partition("※コリンズで") # 文字列の前をタプルとして抽出 
  text1 =t1[0]

else:
  text1 = "記載なし"

doushu_co = text1


#　同種工事の要件文字列の抽出（技術者） doushu_en
if (("工事に専任で配置できること" in text) 
    and 
    (("に掲げる同種工事の要件を満たす工事現場に従事した経験を有する者") in text )
    or ("に掲げる要件の施工経験を有する者であること。")):
  # 技術者への同種工事の経験を有する記載があるかの判定
  doushu_en = doushu_co
else:
  doushu_en = "記載なし"


print("「同種工事（企業）」", doushu_co)
print("「同種工事（技術者）」", doushu_en)


# より同種性の高い工事／同種性が認められる工事の内容の抽出
# 2)評価基準及び得点配分 ①－１企業の能力等（加算点１）及び
# ②配置予定技術者の能力等（加算点２）の表外の記載からそれぞれの内容を抽出する
# doushusei_co01 企業での「より高い同種性の高い工事」の内容
# doushusei_co02 企業での「同種工事」の内容 
# doushusei_en01 技術者での「より高い同種性の高い工事」の内容
# doushusei_en02 技術者での「同種工事」の内容
# 配点は表内にマトリックスで記載されているが抽出方法は要検討
# score_en 技術者での同種工事の施工実績での配点点数 (str)　# 要検討：表内抽出
# score_co 企業での同種工事の施工実績での配点点数 (str)  　# 要検討：表内抽出


if name_of != "本官": # 本官以外の場合
  if (("同種工事とは" in text) and ("より同種工事とは" in text)):
    if ("同種工事：" in text) and ("より同種工事：" in text):
    # 上記文字列で表外記載がある場合の処理ルーチン
      t0 = text0.partition("同種工事とは４") # "企業の能力等（加算点１）"後の文字列をタプルとして抽出
      text0 = t0[-1]
      t1 = text0.partition("同種工事：")  # "）に明示しているとおりである。"の後の文字列をタプルとして抽出
      text1 = t1[-1]
      t1 = text1.partition("◇より同種工事とは") # 文字列の前をタプルとして抽出 
      text1 = t1[0]
      doushu_co02 = text1
      t0 = text0.partition("より同種工事とは、") # 後の文字列を抽出
      text0 = t0[-1]
      t1 = text0.partition("より同種工事：") # 後の文字列をタプルとして抽出
      text1 = t1[-1]
      t1 = text1.partition("◇施工実績") # 前の文字列をタプルとして抽出
      text1 = t1[0]
      doushu_co01 = text1
     
    else:
      t0 = text0.partition("同種工事とは４") # "企業の能力等（加算点１）"後の文字列をタプルとして抽出
      text0 = t0[-1]
      t1 = text0.partition("）に明示しているとおりである。・")  # "）に明示しているとおりである。"の後の文字列をタプルとして抽出
      text1 = t1[-1]
      t1 = text1.partition("◇より同種工事とは") # 文字列の前をタプルとして抽出 
      text1 = t1[0]
      doushu_co02 = text1
      t0 = text0.partition("より同種工事とは、") # 後の文字列を抽出
      text0 = t0[-1]
      t1 = text0.partition("次のとおりとする。・") # 後の文字列をタプルとして抽出
      text1 = t1[-1]
      t1 = text1.partition("◇施工実績") # 前の文字列をタプルとして抽出
      text1 = t1[0]
      doushu_co01 = text1

else: # 本官工事の場合
  if ("同種工事：" in text) and ("より同種工事：" in text):
  # 上記文字列で表外記載がある場合の処理ルーチン
    t0 = text0.partition("同種工事とは４") # "企業の能力等（加算点１）"後の文字列をタプルとして抽出
    text0 = t0[-1]
    t1 = text0.partition("同種工事：")  # "）に明示しているとおりである。"の後の文字列をタプルとして抽出
    text1 = t1[-1]
    t1 = text1.partition("◇より同種工事とは") # 文字列の前をタプルとして抽出 
    text1 = t1[0]
    doushu_co02 = text1
    t0 = text0.partition("より同種工事とは、") # 後の文字列を抽出
    text0 = t0[-1]
    t1 = text0.partition("より同種工事：") # 後の文字列をタプルとして抽出
    text1 = t1[-1]
    t1 = text1.partition("◇施工実績") # 前の文字列をタプルとして抽出
    text1 = t1[0]
    doushu_co01 = text1 

if ("配置予定技術者の能力等（加算点２）" in text0):
  if("配置予定技術者に関する発注機関別の考え方は、施工実績に準ずる"):
    doushu_en01 = doushu_co01
    doushu_en02 = doushu_co02


print("「より同種性の高い（企業）」" ,doushu_co01)
print("「同種工事（企業）」" ,doushu_co02)
print("「より同種性の高い（技術者）」" ,doushu_en01)
print("「同種工事（技術者）」" ,doushu_en02)


【入札説明書】令和5年度北部国道管内保全(その１)工事.txt
39615
True

False
北部国道事務所
令和５年度北部国道管内保全（その１）工事
「同種工事（企業）」 道路における維持修繕工事の施工実績。（道路とは、道路法に基づく国道（高速自動車道含む）・県市町村道をいう。）
「同種工事（技術者）」 道路における維持修繕工事の施工実績。（道路とは、道路法に基づく国道（高速自動車道含む）・県市町村道をいう。）
「より同種性の高い（企業）」 道路における維持修繕工事で防護柵を設置した施工実績。（道路とは、道路法に基づく国道（高速自動車道含む）・県市町村道をいう。）
「同種工事（企業）」 道路における維持修繕工事の施工実績。（道路とは、道路法に基づく国道（高速自動車道含む）・県市町村道をいう。）
「より同種性の高い（技術者）」 道路における維持修繕工事で防護柵を設置した施工実績。（道路とは、道路法に基づく国道（高速自動車道含む）・県市町村道をいう。）
「同種工事（技術者）」 道路における維持修繕工事の施工実績。（道路とは、道路法に基づく国道（高速自動車道含む）・県市町村道をいう。）


### ８１北海道開発局：テキスト文字列からの抽出「同種性」
- 抽出元：入札説明書（表外の表記より）
- １０開発建設部（札幌、函館、小樽、旭川、室蘭、釧路、帯広、網走、留萌、稚内）毎でのサブルーチン（テンプレの表記ゆれ対応）


In [ ]:
# 810 北海道開発局札幌開発建設部：入札説明書（PDF）から「より同種性が高い」工事要件を抽出するテストプログラム
# 別ファイルである「別表.pdf」のうち、イ　企業評価について　の表内表記から文字列を抽出する必要あり
# 別ファイル未対応
import re


# inputfile = "1157入札説明書.txt"
# inputfile = "2047入札説明書.txt"
# inputfile = "2048入札説明書.txt"
# inputfile ="2050入札説明書.txt"
# inputfile = "８６０６　入札説明書　一般国道２３１号　石狩市　嶺泊スノーシェルター補修外一連工事.txt"
inputfile ="８６１０　入札説明書　道央圏連絡道路　南幌町　南７線舗装外一連工事.txt"



with open(inputfile, "rt", encoding="utf-8") as f:  # 改行コード等除去後のテキストファイルを開く
  text = f.read()

print(inputfile)
print(len(text))
print("総合評価に関する事項" in text)

# 整備局等名・事務所名の抽出
# namu_bu 地方整備局等
# name_of 事務所名（本官工事の場合は「本官」）

if ("北海道開発局札幌開発建設部" in text):
  name_bu =""

print(name_bu)

print ("支出負担行為担当官北海道開発局札幌開発建設部長" in text)  # 本官と分任官とのルーチン分け
if ("支出負担行為担当官北海道開発局札幌開発建設部長" in text):  # 本官と分任官とのルーチン分け
  name_of = "北海道開発局札幌開発建設部"
else:  # 事務所名抽出：１つめの分任支出負担行為担当官以下の　所長までの部分でスライス
  t0 = text.partition("分任支出負担行為担当官")  # 入札説明書の１つめの同一文字列の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("北海道開発局札幌開発建設部") # ”北海道開発局札幌開発建設部”の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("所長")                  # "所長"の文字列の前をタプルとして抽出
  text0 = t0[0]
  name_of = text0.strip() + "所"              # 冒頭空白文字を削除して、抽出した文字列に"所"を追加

print(name_of)

# 工事名の抽出
# name_co  工事名 
# (1) 工事名 , (2) 工事場所 で文字列を抽出スライス
if ("(電子入札対象案件)" in text): # ()の半角・全角処理
  name_end = "(電子入札対象案件)"
elif ("（電子入札対象案件）" in text):
  name_end = "（電子入札対象案件）"

t0 = text.partition("工事名")  # "工事名"の文字列の後をタプルとして抽出
text0 = t0[-1]
t0 = text0.partition("(2)")    # 次行にある(2)の文字列の前をタプルとして抽出
text0 = t0[0]
t0 = text0.partition(name_end)  # "（電子入札対象案件）"文字列の前をタプルとして抽出
text0 = t0[0]
name_co =text0.strip()         # 前後の空白文字を削除

print(name_co)

# 同種工事の要件文字列の抽出（企業） doushu_co
# より同種性の高い工事が記載されている場合
# doushu_co01 :「より同種性の高い工事」
# doushu_co02 :「同種工事」または「同種性が認められる工事」

print("" in text)
if (("施工した実績を有すること" in text) and ("当該実績が北海道開発局、" in text)):
  # （４）工事実績の要件記載があるかの判定
  t0 = text.partition("点未満のものを除く。")   # 後の文字列をタプルとして抽出
  text0 = t0[-1]
  if (("①【同種条件】" in text0) and ("②【より同種性の高い" in text0)):
    # ①【同種条件】、②【より同種性の高い工事条件】と列記している場合
    t1 = text0.partition("①【同種条件】") # 後の文字列をタプルとして抽出
    text1 = t1[-1]
    t1 = text1.partition("(施工実績が確認") # 文字列の前をタプルとして抽出
    text1 = t1[0]
    doushu_co = text1
    doushu_co02 = text1

    t1 = text0.partition("②【より同種性の高い工事条件】") # 後の文字列をタプルとして抽出
    text1 = t1[-1]
    t1 = text1.partition("(施工実績が確認") # 文字列の前をタプルとして抽出
    text1 = t1[0]
    doushu_co01 = text1

  elif ("①【同種条件】" in text0):
    # ①【同種条件】のみ列記している場合
    t1 = text0.partition("①【同種条件】") # 後の文字列をタプルとして抽出
    text1 = t1[-1]
    t1 = text1.partition("(施工実績が確認") # 文字列の前をタプルとして抽出
    text1 = t1[0]
    doushu_co = text1
    doushu_co01 = ""
    doushu_co02 = ""
  else:
    doushu_co = "同種条件の記載なし"
else:
  doushu_co = "工事実績の要件記載なし"

if ("上記(4)本文に掲げる工事の経験を有する者であること" in text):
  doushu_en = doushu_co
elif (("上記(4)本文に掲げる工事を" in text) and "を有する者であること" in text):
  doushu_en = doushu_co

else:
  doushu_en = "工事実績の要件記載なし"

# 技術者での同種工事、より同種性の高い工事の評価については、
# 別表の他のPDFファイルを参照する必要あり（未対応）

print("「同種工事（企業）」" ,doushu_co)
print("「同種工事（技術者）」" ,doushu_en)
print("「より同種性の高い工事（企業）」" ,doushu_co01)
print("「同種工事（企業）", doushu_co02)

In [ ]:
# 未対応の開発建設部
#
# 810 北海道開発局札幌開発建設部
# 入札説明書（PDF）から「より同種性が高い（企業）」までは抽出可能だが、技術者以降の条件は
# 別ファイルである「別表.pdf」のうち、イ　企業評価について　の表内表記から文字列判別をする必要あり
# 811 北海道開発局函館開発建設部
# 入札説明書（PDF）だけでなく、別ファイルとなる評価基準02.pdfの表内表記より抽出する必要あり。
# （同種工事までなら入札説明書で可）
# 812 北海道開発局小樽開発建設部
# ※入札説明書が一太郎ファイル（jtd）のため、別モジュールでテキスト化する必要あり。
　

In [ ]:
# 813 北海道開発局旭川開発建設部：入札説明書（PDF）から「より同種性が高い」工事要件を抽出するテストプルグラム
# 点数配分は入札説明書の後半の別表１アの表内表記より抽出する必要あり。
import re

# inputfile ="【2231】多寄舗装工事_入札説明書.txt"
# inputfile ="【2286】_一般国道３９号　比布町　比布大橋下部工事_入札説明書.txt"
# inputfile = "2010_一般国道４０号　音威子府村　音中トンネル電気室Ａ棟新築工事_入札説明書.txt"
# inputfile ="2021_一般国道４０号　中川町　音中トンネル電気室Ｂ棟新築工事_入札説明書.txt"
# inputfile ="2112_石狩川改修工事の内　辺別川左岸堤防強化外工事_入札説明書.txt"
# inputfile ="2245_富良野電線共同溝設置工事_入札説明書(new).txt"
inputfile ="2253_中央舗装補修外一連工事_入札説明書.txt"


with open(inputfile, "rt", encoding="utf-8") as f:  # 改行コード等除去後のテキストファイルを開く
  text = f.read()

print(inputfile)
print(len(text))
print("総合評価に関する事項" in text)

# 整備局等名・事務所名の抽出
# namu_bu 地方整備局等
# name_of 事務所名（本官工事の場合は「本官」）

if ("北海道開発局旭川開発建設部" in text):
  name_bu =""

print(name_bu)

print ("支出負担行為担当官旭川開発建設部長" in text)  # 本官と分任官とのルーチン分け
if ("支出負担行為担当官旭川開発建設部長" in text):  # 本官と分任官とのルーチン分け
  name_of = "北海道開発局旭川開発建設部"
else:  # 事務所名抽出：１つめの分任支出負担行為担当官以下の　所長までの部分でスライス
  t0 = text.partition("分任支出負担行為担当官")  # 入札説明書の１つめの同一文字列の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("旭川開発建設部") # ”北海道開発局旭川開発建設部”の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("所長")                  # "所長"の文字列の前をタプルとして抽出
  text0 = t0[0]
  name_of = text0.strip() + "所"              # 冒頭空白文字を削除して、抽出した文字列に"所"を追加

print(name_of)

# 工事名の抽出
# name_co  工事名 
# (1) 工事名 , (2) 工事場所 で文字列を抽出スライス

t0 = text.partition("工事名")  # "工事名"の文字列の後をタプルとして抽出
text0 = t0[-1]
t0 = text0.partition("（2）")    # 次行にある(2)の文字列の前をタプルとして抽出
text0 = t0[0]
t0 = text0.partition("（電子入札対象案件）")  # "（電子入札対象案件）"文字列の前をタプルとして抽出
text0 = t0[0]
name_co =text0.strip()         # 前後の空白文字を削除

print(name_co)

# 同種工事の要件文字列の抽出（企業） doushu_co
# より同種性の高い工事が記載されている場合
# doushu_co01 :「より同種性の高い工事」
# doushu_co02 :「同種工事」または「同種性が認められる工事」

if (("（以下「同種工事の施工実績」という。）を有すること" in text)
    and ("当該実績が北海道開発局、" in text)):
  # （４）工事実績の要件記載があるかの判定
  t0 = text.partition("点未満のものを除く。")   # 後の文字列をタプルとして抽出
  text0 = t0[-1]
  if (("同種性が認められる工事" in text) and ("より同種性の高い工事" in text)):
    # ア〔同種性が認められる工事〕、イ〔より同種性の高い工事〕と列記している場合
    t1 = text0.partition("〔同種性が認められる工事〕") # 後の文字列をタプルとして抽出
    text1 = t1[-1]
    t1 = text1.partition("イ〔") # 文字列の前をタプルとして抽出
    text1 = t1[0]
    doushu_co = text1
    doushu_co02 = text1

    t1 = text0.partition("〔より同種性の高い工事〕") # 後の文字列をタプルとして抽出
    text1 = t1[-1]
    t1 = text1.partition("（５）") # 文字列の前をタプルとして抽出
    text1 = t1[0]
    doushu_co01 = text1
  elif ("同種性が認められる工事" in text):
    # ア〔同種性が認められる工事〕のみ列記している場合
    t1 = text0.partition("〔同種性が認められる工事〕") # 後の文字列をタプルとして抽出
    text1 = t1[-1]
    t1 = text1.partition("（５）") # 文字列の前をタプルとして抽出
    text1 = t1[0]
    doushu_co = text1
    doushu_co01 = ""
    doushu_co02 = text1
  else:
    doushu_co = ""
else:
  doushu_co = ""

# 技術者での参加資格要件
if ("上記(４)に掲げる工事の経験を有する者であること" in text):
  doushu_en = doushu_co
else:
  doushu_en = ""


# 同種工事の要件文字列の判定（技術者）
# 別表１　ア　評価項目等について　（加点項目　の表内において、配置予定技術者としての評価基準の記載がある場合に設定
# doushu_en01 :「より同種性の高い工事」（役職）「同種性が認められる工事」（役職）
# doushu_en02 :「より同種性の高い工事」（役職）「同種性が認められる工事」（役職）
# doushu_en03：「同種性が認められる工事」（役職）
# なお、表中の文字列抽出は行っておらず、各総合評価型での一般的な記載例を設定している。

t0 = text0.partition("当該工事を担当する営業所等に関するものを提出すること。")
text0 = t0[-1]

if ("同種性が認められる工事の従事実績なし" in text0): # 同種性が認められる工事の役職の２段階で評価する場合
  if (("において、監理（主任）技術者、特例監理" in text0) 
      and ("において、監理技術者補佐又は担当技術者として従事" in text0)):
    doushu_en01 = "同種性が認められる工事において、監理（主任）技術者、特例監理技術者又は現場代理人として従事"
    doushu_en02 = "同種性が認められる工事において、監理技術者補佐又は担当技術者として従事"
    doushu_en03 = ""
elif (("より同種性の高い工事において、監理" in text0) 
      and ("補佐又は担当技術者として従事" in text0) 
      and ("同種性が認められる工事において、監理" in text0)):
  doushu_en01 = "実績より同種性の高い工事において、監理(主任) 技術者、特例監理技術者又は現場代理人として従事"
  doushu_en02 = "より同種性の高い工事において、監理技術者補佐又は担当技術者として従事、または同種性が認められる工事において、監理(主任) 技術者、特例監理技術者又は現場代理人として従事"
  doushu_en03 = "同種性が認められる工事において、監理技術者補佐又は担当技術者として従事"
else:
  doushu_en01 = ""
  doushu_en02 = ""
  doushu_en03 = ""

print("「同種工事（企業）」" ,doushu_co)
print("「同種工事（技術者）」" ,doushu_en)
print("「より同種性の高い工事（企業）」" ,doushu_co01)
print("「同種工事（企業）」", doushu_co02)
print("「より同種性の高い工事（技術者）」" ,doushu_en01)
print("「より同種性の高い工事・同種工事（技術者）」", doushu_en02)
print("「同種工事（技術者）」", doushu_en03)



In [ ]:
# 814 北海道開発局室蘭開発建設部：入札説明書（PDF）から「より同種性が高い」工事要件を抽出するテストプログラム
# ★★★　そのままPyMuPDFでテキスト変換すると抽出テキストの文章が　★★★
# ★★★　バラバラの状態となるため、一度ＯＣＲで変換する必要あり。　★★★
# 点数配分は入札説明書の後半の別表１の表内表記より抽出する必要あり。
import re

# inputfile ="T2601_nyuusetu01.txt"
# inputfile ="T2602_nyuusetu01.txt"
# inputfile ="T2603_nyuusetu01.txt"
# inputfile ="T2604_nyuusetu01.txt"
# inputfile ="T2605_nyuusetu01.txt"
# inputfile ="T2606_nyuusetu01.txt"
inputfile ="T2607_nyuusetu01.txt"


with open(inputfile, "rt", encoding="utf-8") as f:  # 改行コード等除去後のテキストファイルを開く
  text = f.read()

print(inputfile)
print(len(text))
print("総合評価に関する事項" in text)

# 整備局等名・事務所名の抽出
# namu_bu 地方整備局等
# name_of 事務所名（本官工事の場合は「本官」）

if ("北海道開発局室蘭開発建設部" in text):
  name_bu =""

print(name_bu)

if (("支出負担行為担当官" in text) and ("北海道開発局室蘭開発建設部長" in text)):  # 本官と分任官とのルーチン分け
  name_of = "北海道開発局室蘭開発建設部"
else:  # 事務所名抽出：１つめの分任支出負担行為担当官以下の　所長までの部分でスライス
  t0 = text.partition("分任支出負担行為担当官")  # 入札説明書の１つめの同一文字列の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("北海道開発局札幌開発建設部") # ”北海道開発局札幌開発建設部”の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("所長")                  # "所長"の文字列の前をタプルとして抽出
  text0 = t0[0]
  name_of = text0.strip() + "所"              # 冒頭空白文字を削除して、抽出した文字列に"所"を追加

print(name_of)

# 工事名の抽出
# name_co  工事名 
# (1) 工事名 , (2) 工事場所 で文字列を抽出スライス

t0 = text.partition("工 事 名")  # "工事名"の文字列の後をタプルとして抽出
text0 = t0[-1]
t0 = text0.partition("(電子入札対象案件)")  # "（電子入札対象案件）"文字列の前をタプルとして抽出
text0 = t0[0]
name_co =text0.strip()         # 前後の空白文字を削除

print(name_co)

# 同種工事の要件文字列の抽出（企業） doushu_co
# より同種性の高い工事が記載されている場合
# doushu_co01 :「より同種性の高い工事」
# doushu_co02 :「同種工事」または「同種性が認められる工事」

if (("施工した実績を有すること" in text) 
    and ("当該実績が北海道開発局、" in text)):
  # （４）工事実績の要件記載があるかの判定
  t0 = text.partition("点未満のものを除く。")   # 後の文字列をタプルとして抽出
  text0 = t0[-1]
  if (("より同種性の高い工事" in text0) and ("同種性が認められる工事" in text0)):
    # より同種性の高い工事、同種性が認められる工事と列記している場合
    t1 = text0.partition("・より同種性の高い工事：") # 後の文字列をタプルとして抽出
    text1 = t1[-1]
    t1 = text1.partition("・同種性") # 文字列の前をタプルとして抽出
    text1 = t1[0]
    text1 = text1.replace("　","") # 空白文字の除去
    doushu_co01 = text1
    
    t1 = text0.partition("・同種性が認められる工事：") # 後の文字列をタプルとして抽出
    text1 = t1[-1]
    t1 = text1.partition("(５)") # 文字列の前をタプルとして抽出
    text1 = t1[0]
    text1 = text1.replace("　","") # 空白文字の除去
    doushu_co = text1
    doushu_co02 = text1

  else:
    doushu_co = ""
else:
  doushu_co = ""

if ("上記(４)に掲げる工事の経験を有する者であること" in text):
  doushu_en = doushu_co
else:
  doushu_en = ""

# 同種工事の要件文字列の判定（技術者）
# (2)評価の基準　ア　入札の評価に関する基準　の表内において、配置予定技術者としての評価基準の記載がある場合に設定
# doushu_en01 :「より同種性の高い工事」（役職）
# doushu_en02 :「より同種性の高い工事」（役職）「同種性が認められる工事」（役職）
# doushu_en03：「同種性が認められる工事」（役職）
# なお、表中の文字列抽出は行っておらず、各総合評価型での一般的な記載例を設定している。

t0 = text0.partition("下記の評価基準に基づき加点する")
text0 = t0[-1]

if (("より同種性の高い工事において、監理" in text0) #より同種性の高い工事において、監理
    and ("補佐又は担当技術者として従事、ま" in text0) 
    and ("同種性が認められる工事において、監理" in text0)):
  doushu_en01 = "実績より同種性の高い工事において、監理(主任) 技術者、特例監理技術者又は現場代理人として従事"
  doushu_en02 = "より同種性の高い工事において、監理技術者補佐又は担当技術者として従事、または同種性が認められる工事において、監理(主任) 技術者、特例監理技術者又は現場代理人として従事"
  doushu_en03 = "同種性が認められる工事において、監理技術者補佐又は担当技術者として従事"
else:
  doushu_en01 = ""
  doushu_en02 = ""
  doushu_en03 = ""

print("「同種工事（企業）」" ,doushu_co)
print("「同種工事（技術者）」" ,doushu_en)
print("「より同種性の高い工事（企業）」" ,doushu_co01)
print("「同種工事（企業）」", doushu_co02)
print("「より同種性の高い工事（技術者）」" ,doushu_en01)
print("「より同種性の高い工事・同種工事（技術者）」", doushu_en02)
print("「同種工事（技術者）」", doushu_en03)



In [ ]:
# 815 北海道開発局釧路開発建設部：入札説明書（PDF）から「より同種性が高い」工事要件を抽出するテストプログラム
# 評価配点については、後段の　評価(2) 評価の基準　ア評価項目等について　の表内の表記を抽出する必要あり
# なお、R5.4 以前の入札説明書からはテキスト抽出が上手くいかずに抽出できていない工事がある。
import re

# inputfile = "2096 一般国道４４号　根室市　川口中央帯舗装工事.txt"
# inputfile ="2097 一般国道３３５号　標津町　崎無異道路改良外一連工事.txt"
# inputfile ="2102 一般国道４４号　釧路町　雪裡橋右岸仮橋設置工事.txt"
# inputfile = "2105 一般国道３８号　釧路市　鶴野改良工事.txt"
# inputfile = "2106 北海道横断自動車道　釧路市　桜田舗装工事.txt"
# inputfile = "2107 北海道横断自動車道　釧路市　山花東舗装工事.txt"
# inputfile = "2113 一般国道４４号　厚岸町　太田宏陽改良外一連工事.txt"
# inputfile ="2114 一般国道４４号　厚岸町　門静改良外一連工事.txt"
inputfile = "2117 釧路川改修工事の内　下鐺別橋下流河道整備工事.txt"



with open(inputfile, "rt", encoding="utf-8") as f:  # 改行コード等除去後のテキストファイルを開く
  text = f.read()

print(inputfile)
print(len(text))
print("総合評価に関する事項" in text)

# 整備局等名・事務所名の抽出
# namu_bu 地方整備局等
# name_of 事務所名（本官工事の場合は「本官」）

if ("北海道開発局札幌開発建設部" in text):
  name_bu =""

print(name_bu)

print ("支出負担行為担当官北海道開発局釧路開発建設部長" in text)  # 本官と分任官とのルーチン分け
if ("支出負担行為担当官北海道開発局釧路開発建設部長" in text):  # 本官と分任官とのルーチン分け
  name_of = "北海道開発局釧路開発建設部"
else:  # 事務所名抽出：１つめの分任支出負担行為担当官以下の　所長までの部分でスライス
  t0 = text.partition("分任支出負担行為担当官")  # 入札説明書の１つめの同一文字列の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("北海道開発局釧路開発建設部") # ”北海道開発局釧路開発建設部”の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("所長")                  # "所長"の文字列の前をタプルとして抽出
  text0 = t0[0]
  name_of = text0.strip() + "所"              # 冒頭空白文字を削除して、抽出した文字列に"所"を追加

print(name_of)

# 工事名の抽出
# name_co  工事名 
# (1) 工事名 , (2) 工事場所 で文字列を抽出スライス

t0 = text.partition("工事名")  # "工事名"の文字列の後をタプルとして抽出
text0 = t0[-1]
t0 = text0.partition("(2)")    # 次行にある(2)の文字列の前をタプルとして抽出
text0 = t0[0]
t0 = text0.partition("（電子入札対象案件）")  # "（電子入札対象案件）"文字列の前をタプルとして抽出
text0 = t0[0]
name_co =text0.strip()         # 前後の空白文字を削除

print(name_co)


# 同種工事の要件文字列の抽出（企業） doushu_co
# より同種性の高い工事が記載されている場合
# doushu_co01 :「より同種性の高い工事」
# doushu_co02 :「同種性が認められる工事」

if (("施工した実績を有すること" in text) 
    and ("当該実績が北海道開発局、" in text)):
  # （４）工事実績の要件記載があるかの判定
  t0 = text.partition("点未満のものを除く。")   # 後の文字列をタプルとして抽出
  text0 = t0[-1]
  if (("同種性が認められる工事" in text0) and ("より同種性の高い工事" in text0)):
    # ①【同種条件】、②【より同種性の高い工事条件】と列記している場合
    t1 = text0.partition("同種性が認められる工事・") # 後の文字列をタプルとして抽出
    text1 = t1[-1]
    t1 = text1.partition("イ") # 文字列の前をタプルとして抽出
    text1 = t1[0]
    doushu_co = text1
    doushu_co02 = text1

    t1 = text0.partition("より同種性の高い工事・") # 後の文字列をタプルとして抽出
    text1 = t1[-1]
    t1 = text1.partition("(5)") # 文字列の前をタプルとして抽出
    text1 = t1[0]
    doushu_co01 = text1

  else:
    doushu_co = "同種条件の記載なし"
else:
  doushu_co = "工事実績の要件記載なし"

if ("上記(4)に掲げる工事の経験を有する者であること" in text):
  doushu_en = doushu_co
else:
  doushu_en = "工事実績の要件記載なし"

# 同種工事の要件文字列の判定（技術者）
# (2)評価の基準　ア　入札の評価に関する基準　の表内において、配置予定技術者としての評価基準の記載がある場合に設定
# doushu_en01 :「より同種性の高い工事」（役職）
# doushu_en02 :「より同種性の高い工事」（役職）「同種性が認められる工事」（役職）
# doushu_en03：「同種性が認められる工事」（役職）
# なお、表中の文字列抽出は行っておらず、各総合評価型での一般的な記載例を設定している。

t0 = text0.partition("評価基準に基づき加点する。")
text0 = t0[-1]

if (("より同種性の高い工事において、監理" in text0) #より同種性の高い工事において、監理
    and ("補佐又は担当技術者として従事、ま" in text0) 
    and ("同種性が認められる工事において、監理" in text0)):
  doushu_en01 = "実績より同種性の高い工事において、監理(主任) 技術者、特例監理技術者又は現場代理人として従事"
  doushu_en02 = "より同種性の高い工事において、監理技術者補佐又は担当技術者として従事、または同種性が認められる工事において、監理(主任) 技術者、特例監理技術者又は現場代理人として従事"
  doushu_en03 = "同種性が認められる工事において、監理技術者補佐又は担当技術者として従事"
else:
  doushu_en01 = ""
  doushu_en02 = ""
  doushu_en03 = ""

print("「同種工事（企業）」" ,doushu_co)
print("「同種工事（技術者）」" ,doushu_en)
print("「より同種性の高い工事（企業）」" ,doushu_co01)
print("「同種工事（企業）」", doushu_co02)
print("「より同種性の高い工事（技術者）」" ,doushu_en01)
print("「より同種性の高い工事・同種工事（技術者）」", doushu_en02)
print("「同種工事（技術者）」", doushu_en03)


In [ ]:
# 816 北海道開発局帯広開発建設部：入札説明書（PDF）から「より同種性が高い」工事要件を抽出するテストプログラム
# ★★★　そのままPyMuPDFでテキスト変換すると抽出テキストの文章が　★★★
# ★★★　バラバラの状態となるため、一度ＯＣＲで変換する必要あり。　★★★
# 点数配分は入札説明書の(2) 評価の基準　ア 入札の評価に関する基準　の表内表記より抽出する必要あり。
import re

# inputfile ="T871 入札説明書.txt"
# inputfile ="T872 入札説明書.txt"
# inputfile ="T1004 入札説明書.txt"
inputfile ="T2801_A setumei.txt"



with open(inputfile, "rt", encoding="utf-8") as f:  # 改行コード等除去後のテキストファイルを開く
  text = f.read()

print(inputfile)
print(len(text))
print("総合評価に関する事項" in text)

# 整備局等名・事務所名の抽出
# namu_bu 地方整備局等
# name_of 事務所名（本官工事の場合は「本官」）

if ("北海道開発局帯広開発建設部" in text):
  name_bu =""

print(name_bu)

if (("支出負担行為担当官" in text) and ("帯広開発建設部長" in text)):  # 本官と分任官とのルーチン分け
  name_of = "北海道開発局帯広開発建設部"
else:  # 事務所名抽出：１つめの分任支出負担行為担当官以下の　所長までの部分でスライス
  t0 = text.partition("分任支出負担行為担当官")  # 入札説明書の１つめの同一文字列の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("北海道開発局札幌開発建設部") # ”北海道開発局札幌開発建設部”の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("所長")                  # "所長"の文字列の前をタプルとして抽出
  text0 = t0[0]
  name_of = text0.strip() + "所"              # 冒頭空白文字を削除して、抽出した文字列に"所"を追加

print(name_of)

# 工事名の抽出
# name_co  工事名 
# (1) 工事名 , (2) 工事場所 で文字列を抽出スライス

t0 = text.partition("工 事 名")  # "工事名"の文字列の後をタプルとして抽出
text0 = t0[-1]
t0 = text0.partition("( 電子入札対象案件) ")  # "（電子入札対象案件）"文字列の前をタプルとして抽出
text0 = t0[0]
name_co =text0.strip()         # 前後の空白文字を削除

print(name_co)

# 同種工事の要件文字列の抽出（企業） doushu_co
# より同種性の高い工事が記載されている場合
# doushu_co01 :「より同種性の高い工事」
# doushu_co02 :「同種工事」または「同種性が認められる工事」


if (("同種の工事の施工実績」という。) を有すること" in text) 
    and ("当該実績が北海道開発局、" in text)):
  # （４）工事実績の要件記載があるかの判定
  t0 = text.partition("点未満のものを除く。")   # 後の文字列をタプルとして抽出
  text0 = t0[-1]
  t1 = text0.partition("(５)") # 文字列の前をタプルとして抽出
  text1 = t1[0]
  text1 = text1.replace("　","") # 空白文字の除去
  doushu_co = text1
  doushu_co02 = text1

  if ("上記(4)に掲げる工事の経験を有する者であること" in text0):
    doushu_en = text1
  
  if ("より同種性の高い工事とは、" in text0):
    t0 = text0.partition("より同種性の高い工事とは、") # 後の文字列をタプルとして抽出
    text1 = t0[-1]
    t1 = text1.partition("をいう。") # 文字列の前をタプルとして抽出
    text1 = t1[0]
    doushu_co01 = text1
  else:
    doushu_co01 = ""
    doushu_co02 = ""


# 同種工事の要件文字列の判定（技術者）
# (2)評価の基準　ア　入札の評価に関する基準　の表内において、配置予定技術者としての評価基準の記載がある場合に設定
# doushu_en01 :「より同種性の高い工事」（役職）
# doushu_en02 :「より同種性の高い工事」（役職）「同種性が認められる工事」（役職）
# doushu_en03：「同種性が認められる工事」（役職）
# なお、表中の文字列抽出は行っておらず、各総合評価型での一般的な記載例を設定している。

t0 = text0.partition("評価基準に基づき加点する。")
text0 = t0[-1]

if (("において、監理( 主任) 技術者、特例監理技術者" in text0) 
    and ("監理技術者補佐又は担当技術者として従事、又は" in text0) 
    and ("監理技術者補佐又は担当技術者として従事") in text0):
  doushu_en01 = "実績より同種性の高い工事において、監理(主任) 技術者、特例監理技術者又は現場代理人として従事"
  doushu_en02 = "より同種性の高い工事において、監理技術者補佐又は担当技術者として従事、または同種性が認められる工事において、監理(主任) 技術者、特例監理技術者又は現場代理人として従事"
  doushu_en03 = "同種性が認められる工事において、監理技術者補佐又は担当技術者として従事"
else:
  doushu_en01 = ""
  doushu_en02 = ""
  doushu_en03 = ""

print("「同種工事（企業）」" ,doushu_co)
print("「同種工事（技術者）」" ,doushu_en)
print("「より同種性の高い工事（企業）」" ,doushu_co01)
print("「同種工事（企業）」", doushu_co02)
print("「より同種性の高い工事（技術者）」" ,doushu_en01)
print("「より同種性の高い工事・同種工事（技術者）」", doushu_en02)
print("「同種工事（技術者）」", doushu_en03)


In [ ]:
# 817 北海道開発局網走開発建設部：入札説明書（PDF）から「より同種性が高い」工事要件を抽出するテストプログラム
# ★★★　そのままPyMuPDFでテキスト変換すると、「より同種性の高い工事」の　★★★　
# ★★★　テキスト抽出が不可能となるため、一度ＯＣＲで変換する必要あり。　★★★
# 点数配分は、後半部、別紙「評価基準」表内の文字列を抽出する必要あり。
import re


# inputfile ="T781_入札説明書.txt"
# inputfile ="T782_入札説明書.txt"
# inputfile ="T783_入札説明書.txt"
inputfile ="T784_入札説明書.txt"



with open(inputfile, "rt", encoding="utf-8") as f:  # 改行コード等除去後のテキストファイルを開く
  text = f.read()

print(inputfile)
print(len(text))
print("総合評価に関する事項" in text)

# 整備局等名・事務所名の抽出
# namu_bu 地方整備局等
# name_of 事務所名（本官工事の場合は「本官」）

if ("北海道開発局網走開発建設部" in text):
  name_bu =""

print(name_bu)

if (("支出負担行為担当官" in text) and ("網走開発建設部長" in text)):  # 本官と分任官とのルーチン分け
  name_of = "北海道開発局網走開発建設部"
else:  # 事務所名抽出：１つめの分任支出負担行為担当官以下の　所長までの部分でスライス
  t0 = text.partition("分任支出負担行為担当官")  # 入札説明書の１つめの同一文字列の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("北海道開発局網走開発建設部") # ”北海道開発局網走開発建設部”の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("所長")                  # "所長"の文字列の前をタプルとして抽出
  text0 = t0[0]
  name_of = text0.strip() + "所"              # 冒頭空白文字を削除して、抽出した文字列に"所"を追加

print(name_of)

# 工事名の抽出
# name_co  工事名 
# (1) 工事名 , (2) 工事場所 で文字列を抽出スライス

t0 = text.partition("工事名")  # "工事名"の文字列の後をタプルとして抽出
text0 = t0[-1]
t0 = text0.partition("（電子入札対象案件）")  # "（電子入札対象案件）"文字列の前をタプルとして抽出
text0 = t0[0]
name_co =text0.strip()         # 前後の空白文字を削除

print(name_co)


# 同種工事の要件文字列の抽出（企業） doushu_co
# より同種性の高い工事が記載されている場合
# doushu_co01 :「より同種性の高い工事」
# doushu_co02 :「同種工事」または「同種性が認められる工事」


if (("下記の要件を満たす同種工事の施工実績を有すること" in text) 
    and ("当該実績が北海道開発局、" in text)):
  # （４）工事実績の要件記載があるかの判定
  t0 = text.partition("点未満のものを除く。")   # 後の文字列をタプルとして抽出
  text0 = t0[-1]
  t1 = text0.partition("同種工事は") # の文字列をタプルとして抽出
  text1 = t1[-1]
  t1 = text1.partition("(5)") # 文字列の前をタプルとして抽出
  text1 = t1[0]
  text1 = text1.replace("　","") # 空白文字の除去
  doushu_co = text1
  doushu_co02 = text1

  if (("上記(4)本文に掲げる工事の経験" in text0)
      and ("を有する者であること" in text0)):
    doushu_en = text1
  
  t0 = text0.partition("別紙「評価基準」評価項目")
  text0 = t0[-1]

  if ("より同種性の高い工事は以下のとおりとする。" in text0):
    t0 = text0.partition("より同種性の高い工事は以下のとおりとする。") # 後の文字列をタプルとして抽出
    text1 = t0[-1]
    if ("チャレンジ型のため設定なし" in text1):
      doushu_co01 ="チャレンジ型のため設定なし"
    else:
      t1 = text1.partition("ＪＶの場合は") # 文字列の前をタプルとして抽出
      text1 = t1[0]
      doushu_co01 = text1
  else:
    doushu_co01 = ""
    doushu_co02 = ""


# 同種工事の要件文字列の判定（技術者）
# 別紙「評価基準」の表内において、配置予定技術者としての評価基準の記載がある場合に設定
# doushu_en01 :「より同種性の高い工事」（役職）
# doushu_en02 :「より同種性の高い工事」（役職）「同種性が認められる工事」（役職）
# doushu_en03：「同種性が認められる工事」（役職）
# なお、表中の文字列抽出は行っておらず、各総合評価型での一般的な記載例を設定している。

t0 = text.partition("「評価基準」評価項目")
text0 = t0[-1]

if (("において、監理( 主任) 技術者、特例監理技術者" in text0) 
    and ("監理技術者補佐又は担当技術者として従事、又は" in text0) 
    and ("監理技術者補佐又は担当技術者として従事" in text0)):
  doushu_en01 = "実績より同種性の高い工事において、監理(主任) 技術者、特例監理技術者又は現場代理人として従事"
  doushu_en02 = "より同種性の高い工事において、監理技術者補佐又は担当技術者として従事、または同種性が認められる工事において、監理(主任) 技術者、特例監理技術者又は現場代理人として従事"
  doushu_en03 = "同種性が認められる工事において、監理技術者補佐又は担当技術者として従事"
else:
  doushu_en01 = ""
  doushu_en02 = ""
  doushu_en03 = ""

print("「同種工事（企業）」" ,doushu_co)
print("「同種工事（技術者）」" ,doushu_en)
print("「より同種性の高い工事（企業）」" ,doushu_co01)
print("「同種工事（企業）」", doushu_co02)
print("「より同種性の高い工事（技術者）」" ,doushu_en01)
print("「より同種性の高い工事・同種工事（技術者）」", doushu_en02)
print("「同種工事（技術者）」", doushu_en03)


In [ ]:
# 818 北海道開発局留萌開発建設部：入札説明書（PDF）から「より同種性が高い」工事要件を抽出するテストプログラム
# ★★★　そのままPyMuPDFでテキスト変換すると、「より同種性の高い工事」の　★★★　
# ★★★　テキスト抽出が不可能となるため、一度ＯＣＲで変換する必要あり。　★★★
# 点数配分は、後半部、別表１の表内の文字列を抽出する必要あり。
import re

# inputfile ="T320_入札説明書.txt"
# inputfile ="T321_入札説明書.txt"
# inputfile ="T323_入札説明書.txt"
# inputfile ="T329_入札説明書.txt"
# inputfile ="T332_入札説明書.txt"
# inputfile ="T333_入札説明書.txt"
# inputfile ="T334_入札説明書.txt"
# inputfile ="T335_入札説明書.txt"
# inputfile ="T339_入札説明書.txt"
# inputfile ="T341_入札説明書.txt"
inputfile ="T342_入札説明書.txt"


with open(inputfile, "rt", encoding="utf-8") as f:  # 改行コード等除去後のテキストファイルを開く
  text = f.read()

print(inputfile)
print(len(text))
print("総合評価に関する事項" in text)

# 整備局等名・事務所名の抽出
# namu_bu 地方整備局等
# name_of 事務所名（本官工事の場合は「本官」）

if ("北海道開発局留萌開発建設部" in text):
  name_bu =""

print(name_bu)

if (("支出負担行為担当官" in text) and ("留萌開発建設部長" in text)):  # 本官と分任官とのルーチン分け
  name_of = "北海道開発局留萌開発建設部"
else:  # 事務所名抽出：１つめの分任支出負担行為担当官以下の　所長までの部分でスライス
  t0 = text.partition("分任支出負担行為担当官")  # 入札説明書の１つめの同一文字列の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("北海道開発局留萌開発建設部") # ”北海道開発局留萌開発建設部”の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("所長")                  # "所長"の文字列の前をタプルとして抽出
  text0 = t0[0]
  name_of = text0.strip() + "所"              # 冒頭空白文字を削除して、抽出した文字列に"所"を追加

print(name_of)

# 工事名の抽出
# name_co  工事名 
# (1) 工事名 , (2) 工事場所 で文字列を抽出スライス

t0 = text.partition("工 事 名")  # "工事名"の文字列の後をタプルとして抽出
text0 = t0[-1]
t0 = text0.partition("( 電子入札対象案件) ")  # "（電子入札対象案件）"文字列の前をタプルとして抽出
text0 = t0[0]
name_co =text0.strip()         # 前後の空白文字を削除

print(name_co)


# 同種工事の要件文字列の抽出（企業） doushu_co
# より同種性の高い工事が記載されている場合
# doushu_co01 :「より同種性の高い工事」
# doushu_co02 :「同種工事」または「同種性が認められる工事」

if (("の要件を満たす工事を元請として施工した実績を有すること" in text) 
    and ("当該実績が北海道開発局、" in text)):
  # （４）工事実績の要件記載があるかの判定
  t0 = text.partition("同種工事」の実績とする。")   # 後の文字列をタプルとして抽出
  text0 = t0[-1]
  t1 = text0.partition("【同種性が認められる工事】") # 後の文字列をタプルとして抽出
  text1 = t1[-1]
  t1 = text1.partition("イ【より同種性の高い") # 文字列の前をタプルとして抽出
  text1 = t1[0]
  text1 = text1.replace("　","") # 空白文字の除去
  text1 = re.sub(r'。\d', '。', text1) # 文字列末尾にページ番号数字が入っている場合の除去処理
  doushu_co = text1
  doushu_co02 = text1
  
  t1 = text0.partition("【より同種性の高い工事】") # 後の文字列をタプルとして抽出
  text1 = t1[-1]
  t1 = text1.partition("(5)") # 文字列の前をタプルとして抽出
  text1 = t1[0]
  text1 = text1.replace("　","") # 空白文字の除去
  text1 = re.sub(r'。\d', '。', text1) # 文字列末尾にページ番号数字が入っている場合の除去処理
  doushu_co01 = text1

  if (("上記( 4) に掲げる工事の経験を有する者であること" in text0) 
      or ("上記(4)に掲げる工事の経験を有する" in text0)):
    doushu_en = doushu_co

  else:
    doushu_en = ""

else:
  doushu_co = "同種条件の記載なし"
  doushu_co01 = ""
  doushu_co02 = ""


# 同種工事の要件文字列の判定（技術者）
# （別表１）の表内において、配置予定技術者としての評価基準の記載がある場合に設定
# doushu_en01 :「より同種性の高い工事」（役職）
# doushu_en02 :「より同種性の高い工事」（役職）「同種性が認められる工事」（役職）
# doushu_en03：「同種性が認められる工事」（役職）
# なお、表中の文字列抽出は行っておらず、各総合評価型での一般的な記載例を設定している。

t0 = text0.partition("別表１評価項目")
text0 = t0[-1]

if (("において、監理( 主任) 技術者、特例監理技術者" in text0) 
    and ("監理技術者補佐又は担当技術者として従事、または" in text0) 
    and ("監監理技術者補佐又は担当技術者として従事") in text0):
  doushu_en01 = "実績より同種性の高い工事において、監理(主任) 技術者、特例監理技術者又は現場代理人として従事"
  doushu_en02 = "より同種性の高い工事において、監理技術者補佐又は担当技術者として従事、または同種性が認められる工事において、監理(主任) 技術者、特例監理技術者又は現場代理人として従事"
  doushu_en03 = "同種性が認められる工事において、監理技術者補佐又は担当技術者として従事"
else:
  doushu_en01 = ""
  doushu_en02 = ""
  doushu_en03 = ""

print("「同種工事（企業）」" ,doushu_co)
print("「同種工事（技術者）」" ,doushu_en)
print("「より同種性の高い工事（企業）」" ,doushu_co01)
print("「同種工事（企業）」", doushu_co02)
print("「より同種性の高い工事（技術者）」" ,doushu_en01)
print("「より同種性の高い工事・同種工事（技術者）」", doushu_en02)
print("「同種工事（技術者）」", doushu_en03)


In [ ]:
# 819 北海道開発局稚内開発建設部：入札説明書（PDF）から「より同種性が高い」工事要件を抽出するテストプログラム
# 点数配分は、後半部、（別表１）の表内の文字列を抽出する必要あり。
import re

# inputfile = "202331-0014_一般国道40号　豊富町　開源パーキングシェルター補修外一連工事_入札説明書.txt"
# inputfile = "202331-0015_稚内開発建設部管内　管渠清掃工事_入札説明書.txt"
# inputfile = "202331-0017_一般国道40号　稚内市　中央電線共同溝設置工事_入札説明書.txt"
# inputfile = "202331-0018_一般国道２３８号　枝幸町　幌別橋Ａ１橋台設置工事－nyusetsu.txt"
# inputfile = "202331-0020_稚内開発建設部管内　電気設備改修外一連工事-nyusetsu01.txt" #【若手技術者評価型】
# inputfile = "202331-0021_一般国道２３８号　枝幸町　幌別橋Ｐ３橋脚設置工事-nyusetsu02.txt"
inputfile = "202331-0022_一般国道２３８号　枝幸町　岡島歩道設置外一連工事-nyusetsu02.txt"


with open(inputfile, "rt", encoding="utf-8") as f:  # 改行コード等除去後のテキストファイルを開く
  text = f.read()

print(inputfile)
print(len(text))
print("総合評価に関する事項" in text)

# 整備局等名・事務所名の抽出
# namu_bu 地方整備局等
# name_of 事務所名（本官工事の場合は「本官」）

if ("北海道開発局稚内開発建設部" in text):
  name_bu =""

print(name_bu)

if (("支出負担行為担当官" in text) and ("稚内開発建設部長" in text)):  # 本官と分任官とのルーチン分け
  name_of = "北海道開発局稚内開発建設部"
else:  # 事務所名抽出：１つめの分任支出負担行為担当官以下の　所長までの部分でスライス
  t0 = text.partition("分任支出負担行為担当官")  # 入札説明書の１つめの同一文字列の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("北海道開発局稚内開発建設部") # ”北海道開発局稚内開発建設部”の後をタプルとして抽出
  text0 = t0[-1]
  t0 = text0.partition("所長")                  # "所長"の文字列の前をタプルとして抽出
  text0 = t0[0]
  name_of = text0.strip() + "所"              # 冒頭空白文字を削除して、抽出した文字列に"所"を追加

print(name_of)

# 工事名の抽出
# name_co  工事名 
# (1) 工事名 , (2) 工事場所 で文字列を抽出スライス

t0 = text.partition("工事名")  # "工事名"の文字列の後をタプルとして抽出
text0 = t0[-1]
t0 = text0.partition("（電子入札対象案件）")  # "（電子入札対象案件）"文字列の前をタプルとして抽出
text0 = t0[0]
name_co =text0.strip()         # 前後の空白文字を削除

print(name_co)


# 同種工事の要件文字列の抽出（企業） doushu_co
# より同種性の高い工事が記載されている場合
# doushu_co01 :「より同種性の高い工事（企業）」
# doushu_co02 :「同種工事（企業）」または「同種性が認められる工事（企業）」

if (("の要件を満たす工事を元請として施工した実績を有すること" in text) 
    and (("当該実績が北海道開発局、" in text)) or ("北海道開発局長から入札参加資格の決定を受けた" in text)):
  # （４）工事実績の要件記載があるかの判定
  if ("元請として施工した実績を有すること。" in text):
    
    t0 = text.partition("元請として施工した実績を有すること。") # 後の文字列をタプルとして抽出
    text0 = t0[-1]
  elif ("元請として施工した施工実績を有すること。"):
    t0 = text.partition("元請として施工した施工実績を有すること。") # 後の文字列をタプルとして抽出
    text0 = t0[-1]
  t1 = text0.partition("同種性が認められる工事・") # 後の文字列をタプルとして抽出
  text1 = t1[-1]
  t1 = text1.partition("イより同種性の高い") # 文字列の前をタプルとして抽出
  text1 = t1[0]
  text1 = text1.replace("　","") # 空白文字の除去
  doushu_co = text1
  doushu_co02 = text1
  
  t1 = text0.partition("より同種性の高い工事・") # 後の文字列をタプルとして抽出
  text1 = t1[-1]
  if ("なお、当該実績が北海道開発局、" in text1): # 表記揺れ対応
    t1 = text1.partition("なお、当該実績が北海道開発局、") # 文字列の前をタプルとして抽出
    text1 = t1[0]
  else:
    t1 = text1.partition("(5)") #表記揺れ対応
    text1 = t1[0]
  text1 = text1.replace("　","") # 空白文字の除去
  doushu_co01 = text1

  if ("上記(4)ア又はイに掲げる工事の経験を有する者であること" in text0):
    doushu_en = doushu_co
  elif("次の(ｱ)又は(ｲ)に掲げる工事の経験を有する者であること" in text0):
    doushu_en = doushu_co
  else:
    doushu_en = ""



else:
  doushu_co = "同種条件の記載なし"
  doushu_co01 = ""
  doushu_co02 = ""


# 同種工事の要件文字列の判定（技術者）
# （別表１）の表内において、配置予定技術者としての評価基準の記載、
# ならびに表外に「より同種性の高い工事とは」の表記がある場合に設定
# doushu_en01 :「より同種性の高い工事（技術者）」
# doushu_en02 :「同種性が認められる工事（技術者）」

t0 = text0.partition("（得点配分）")
text0 = t0[-1]

if("若手技術者評価型" in text0): # 若手技術者評価型での記載の確認
  if (("において、監理（主任）技術者、特例監理技術者" in text0) 
     and ("担当技術者として従事" in text0)):
    if (("より同種性の高い工事とは、" in text0) and ("入札説明書記載のとおり" in text0)):
      doushu_en01 = "より同種性の高い工事において、監理（主任）技術者、特例監理技術者、現場代理人、監理技術者補佐又は担当技術者として従事"
      doushu_en02 = "同種性が認められる工事において、監理（主任）技術者、特例監理技術者、現場代理人、監理技術者補佐又は担当技術者として従事"
      doushu_en03 = "同種工事で監理（主任）技術者、特例監理技術者、現場代理人、監理技術者補佐、担当技術者としてのいずれにも従事した経験がない" 
elif("より同種性の高い工事とは、" in text0):
  if (("において、監理(主任)技術者、特例監理技術者" in text0) 
      and ("監理技術者補佐又は担当技術者として従事、または" in text0) 
      and ("監理技術者補佐又は担当技術者として従事" in text0)):
    if (("より同種性の高い工事とは、" in text0) and ("入札説明書記載のとおり" in text0)):
      doushu_en01 = "より同種性の高い工事において、監理(主任)技術者、特例監理技術者または現場代理人として従事"
      doushu_en02 = "より同種性の高い工事において、監理技術者補佐又は担当技術者として従事、または同種性が認められる工事において、監理(主任)技術者、特例監理技術者または現場代理人として従事"
      doushu_en03 = "同種性が認められる工事において、監理技術者補佐又は担当技術者として従事"

else:
  doushu_en01 = ""
  doushu_en02 = ""
  doushu_en03 = ""

print("「同種工事（企業）」" ,doushu_co)
print("「同種工事（技術者）」" ,doushu_en)
print("「より同種性の高い工事（企業）」" ,doushu_co01)
print("「同種工事（企業）」", doushu_co02)
print("「より同種性の高い工事（技術者）」" ,doushu_en01)
print("「より同種性の高い工事・同種工事（技術者）」", doushu_en02)
print("「同種工事（技術者）」", doushu_en03)



### 各モジュール・関数のテスト

In [ ]:
import re

s = 'Line1\nLine2\r\nLine3'
print(s)

s_n = ''.join(s.splitlines())
print(s_n)
print(repr(s_n))

s_n2 = re.sub(r'\n[ ]*\n','', s)
print(s_n2)
print(repr(s_n2))


Line1
Line2
Line3
Line1Line2Line3
'Line1Line2Line3'
Line1
Line2
Line3
'Line1\nLine2\r\nLine3'


In [ ]:
import re

s = "- 1 -  と  - 2 - と - ３ - と - 1234 -"
s_e = re.sub(r'- \d{,3} -', '', s)
print(s_e)


  と   と  と - 1234 -


text2 = re.sub(r'\n[ ]*\n','', text) # 改行行除去処理
print(len(text2))
print(re.search("関東地方整備局", text2))

In [ ]:
# PyMuPDF  Page.find_tables のテストコード  (テスト中）
import os
import fitz  # pymupdfライブラリのインポート
import pandas as pd
import numpy as np

inputfile = "【入札説明書】阿武隈川下流江尻下流地区河道掘削等工事.pdf"
# PDFの読み込み
doc = fitz.open(inputfile)

# ページの指定
page = doc[10]

# 表データの検索
tables = page.find_tables()

if tables.tables:
    table_data = tables[0].extract()
    # 列名にするデータを取得
    columns = table_data[0]

    # 表のデータを取得
    data_rows = table_data[1:]

    # データフレームを作成
    df = pd.DataFrame(data_rows, columns=columns)
    
    df1 = df.replace("", np.nan)  # nanに置き換え
    df2 = df1.dropna(how="all")  # すべての列がnanの行を削除
    
    # データフレームをCSVに保存
    df2.to_csv("output.csv", index=False, header=False)

print(df)
# print(df2)

In [ ]:
import os
import re
import io


path = r'output2\①入札説明書_Ｒ６東京国道管内改築保全工事.txt'

with open(path , "r", encoding = "utf-8") as fin1: # ファイル全体を文字列として読み込み
  text1 = fin1.read()

print(type(text1))
print(re.search("同種工事", text1))
print([m.span() for m in re.finditer("同種工事", text1)])
list = re.findall(r'同種工事', text1)
print(list(0))

with open(path, "r", encoding="utf-8") as fin2:  # ファイル全体をリストとして読み込み
  text2 =fin2.readlines()

# print(type(text2))
# print(text2)

# ext  = sub.serch("同種工事",text)


FileNotFoundError: [Errno 2] No such file or directory: 'output2\\①入札説明書_Ｒ６東京国道管内改築保全工事.txt'

PyMuPDF 1.24.9 ドキュメントより

In [ ]:
import sys, pathlib, pymupdf

# テキスト検索
areas = page.search_for("mupdf")


# すべてのドキュメントテキストを抽出する方法

fname = sys.argv[1]  # get document filename
with pymupdf.open(fname) as doc:  # open document
    text = chr(12).join([page.get_text() for page in doc])
# write as a binary file to support non-ASCII characters
pathlib.Path(fname + ".txt").write_bytes(text.encode())


# ドキュメントから表の内容を抽出する方法
# Method Page.find_tables() は、高い表検出精度を備えて、すべてをあなたのために行います。
# その大きな利点は、外部ライブラリの依存関係がないこと、人工知能や機械学習技術を使用する
# 必要がないことです。また、データ分析のためのPythonパッケージである pandas のための
# 統合されたインターフェースも提供します。
# 標準的な状況をカバーする例の Jupyter ノートブックをご覧いただければ幸いです。
# これには、1つのページに複数の表や複数のページにまたがる表の断片を結合するなどの状況が
# 含まれています。

# 抽出したテキストをマークする方法
# ページ上で任意のテキストを検索するための標準的な検索機能があります: Page.search_for() です。
# これは、見つかったテキストを囲む Rect (矩形) オブジェクトのリストを返します。
# これらの四角形は、見つかったテキストを目に見えるようにマークするために自動的に
# 注釈を挿入するのに使用できます。

import sys
import pymupdf

def mark_word(page, text):
    """Underline each word that contains 'text'.
    """
    found = 0
    wlist = page.get_text("words", delimiters=None)  # make the word list
    for w in wlist:  # scan through all words on page
        if text in w[4]:  # w[4] is the word's string
            found += 1  # count
            r = pymupdf.Rect(w[:4])  # make rect from word bbox
            page.add_underline_annot(r)  # underline
    return found

fname = sys.argv[1]  # filename
text = sys.argv[2]  # search string
doc = pymupdf.open(fname)

print("underlining words containing '%s' in document '%s'" % (word, doc.name))

new_doc = False  # indicator if anything found at all

for page in doc:  # scan through the pages
    found = mark_word(page, text)  # mark the page's words
    if found:  # if anything found ...
        new_doc = True
        print("found '%s' %i times on page %i" % (text, found, page.number + 1))

if new_doc:
    doc.save("marked-" + doc.name)
